<a href="https://colab.research.google.com/github/michaelknorr/CHE4071/blob/main/CHE4071_Exothermic_CSTR_(HW1_Part_B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.optimize import root
from scipy.integrate import solve_ivp
import numpy as np

q = 100 #L/min
cai = 1 #mol/L
Ti = 350 #K
V = 100 #L
rho = 1000 #g/L
C = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750 #K
k0 = 7.2e10 #1/min
UA = 5e4 #J/(min K)
Tc0 = 300 #K
Tc = 300 #K
ca0 = 0.5 #mol/L
T0 = 350 #K


tend=10 #min

# Define the Mass and Energy Balances
def balances(vec):
    ca, T = vec
    k = k0 * np.exp(-ER / T)  # Rate constant
    mass_balance = q/V * (cai - ca) - k*ca  # dCa/dt
    energy_balance = q/V * (Ti - T) + (negHr * k * ca) / (rho * C) - (UA / (rho * V * C)) * (T - Tc)  # dT/dt
    return [mass_balance, energy_balance]

# Right-Hand-Side to Include time for solve_ivp
def rhs(t, vec):
    return balances(vec)

# Initial guesses
initial_guess = [ca0, T0]

# Solve for steady-state using root solver
solution = root(balances, initial_guess)
ca_ss, T_ss = solution.x

#Diff Eq Solver using SS conditions as ICs
res = solve_ivp(rhs, (0, tend), [ca_ss, T_ss], method='Radau', dense_output=True)

#Return Outputs
print(f"Steady-state Concentration of A: {ca_ss:.10f} mol/L")
print(f"Steady-state Temperature: {T_ss:.10f} K")



Steady-state Concentration of A: 0.4999182860 mol/L
Steady-state Temperature: 350.0055286902 K


In [ ]:
#Plotting
from plotly.subplots import make_subplots
tplot=np.linspace(0,tend,200)
ca, T = res.sol(tplot)
fig=make_subplots(rows=1, cols=2)
fig.add_scatter(x=tplot, y=T, row=1, col=1, name='T')
fig.add_scatter(x=tplot, y=ca, row=1, col=2, name='Ca')
fig.update_layout(width=800, height=400, template='plotly_dark')